In [12]:
import torch
import argparse
import math
import os
import tqdm


work_dir = '/tmp/txl-05-12_23-33-51'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the best saved model.
with open(os.path.join(work_dir, 'model-best.pt'), 'rb') as f:
    model = torch.load(f)

model = model.to(device)
_ = model.eval()

In [6]:
from pytorch_pretrained_bert import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [8]:
NL = tokenizer.encode('\n')
print(NL)

[198]


In [26]:
data = torch.tensor(NL*70)
# Turn into a batch
data.unsqueeze_(1)
mems = model.init_mems()

tgt_len = data.size(0)
hidden, new_mems = model._forward(data, mems=mems)
pred_hid = hidden[-tgt_len:]
print(hidden.shape, torch.sum(hidden).item(), torch.mean(hidden).item())


torch.Size([70, 1, 500]) 25.866165161132812 0.0007390332757495344


In [37]:
import torch.nn.functional as F

def hidden_to_softmax(model, hidden):
    """Turn a hidden projection into log softmax.
    
    Adapted from utils/proj_adaptive_softmax.py
    """
    self = model.crit
    logit = self._compute_logit(hidden, self.out_layers[0].weight,
                                            self.out_layers[0].bias, self.out_projs[0])
    softmax = F.softmax(logit, dim=-1)
    return softmax

softmax = hidden_to_softmax(model, pred_hid)
print(softmax.shape, softmax[0].sum().item())

torch.Size([70, 1, 50257]) 0.9999994039535522


In [ ]:
## Grab a sample from the last frame, append to result list, append to `data`
## Run through again
## Decode results from result list